In [1]:
import json

In [2]:
from keras.models import model_from_json

Using TensorFlow backend.


In [23]:
from models import *

In [28]:
def get_model(dropout_rate, model_weights_filename):
    print("Creating Model...")
    metadata = get_metadata()
    num_classes = len(metadata['ix_to_ans'].keys())+1
    num_words = len(metadata['ix_to_word'].keys())

    embedding_matrix = prepare_embeddings(num_words, embedding_dim, metadata)
    model = vqa_model(embedding_matrix, num_words, embedding_dim, seq_length, dropout_rate, num_classes)
    if os.path.exists(model_weights_filename):
        print("Loading Weights...")
        model.load_weights(model_weights_filename)

    return model

In [29]:
model = get_model(0.0,'/home/ubuntu/data_processed/data/model_weights_xception.h5')

Creating Model...
Creating image model...
Creating text model...


/home/ubuntu/data_processed/models.py:11: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(trainable=True, units=512, return_sequences=True, input_shape=(26, 300))`
  model.add(LSTM(output_dim=512, return_sequences=True, input_shape=(seq_length, embedding_dim),trainable=True))
/home/ubuntu/data_processed/models.py:13: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(trainable=True, units=512, return_sequences=False)`
  model.add(LSTM(output_dim=512, return_sequences=False,trainable=True))


Merging final model...
Loading Weights...


/home/ubuntu/data_processed/models.py:29: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  fc_model.add(Merge([vgg_model, lstm_model], mode='mul'))


In [15]:
model_vqa.compile(loss='categorical_crossentropy', optimizer='rmsprop')

In [17]:
import numpy as np
from keras.utils.np_utils import to_categorical
import json
import h5py
import os

In [18]:
data_img = '/home/ubuntu/data/data_processed/data_img.h5'
data_prepo = '/home/ubuntu/data/data_processed/data_prepro.h5'

In [31]:
def right_align(seq,lengths):
    v = np.zeros(np.shape(seq))
    N = np.shape(seq)[1]
    for i in range(np.shape(seq)[0]):
        v[i][N-lengths[i]:N]=seq[i][0:lengths[i]]
    return v

In [32]:
def get_val_data_final():
    img_data = h5py.File(data_img)
    ques_data = h5py.File(data_prepo)

    img_data = np.array(img_data['images_test'])
    img_pos_train = ques_data['img_pos_test']
    train_img_data = np.array([img_data[_-1,:] for _ in img_pos_train])
    tem = np.sqrt(np.sum(np.multiply(train_img_data, train_img_data), axis=1))
    train_img_data = np.divide(train_img_data, np.transpose(np.tile(tem,(2048,1))))

    ques_train = np.array(ques_data['ques_test'])
    ques_length_train = np.array(ques_data['ques_length_test'])
    ques_train = right_align(ques_train, ques_length_train)

    # Convert all last index to 0, coz embeddings were made that way :/
    for _ in ques_train:
        if 12602 in _:
            _[_==12602] = 0

    val_X = [train_img_data, ques_train]

    return val_X

In [33]:
val_X = get_val_data_final()

In [35]:
val_X

[array([[  1.17133798e-03,   2.93803322e-05,   0.00000000e+00, ...,
           3.06069292e-02,   1.02472061e-02,   2.67547224e-02],
        [  1.17133798e-03,   2.93803322e-05,   0.00000000e+00, ...,
           3.06069292e-02,   1.02472061e-02,   2.67547224e-02],
        [  1.17133798e-03,   2.93803322e-05,   0.00000000e+00, ...,
           3.06069292e-02,   1.02472061e-02,   2.67547224e-02],
        ..., 
        [  5.16341685e-02,   2.03264328e-03,   1.87685913e-02, ...,
           5.75522990e-02,   0.00000000e+00,   3.20554095e-03],
        [  5.16341685e-02,   2.03264328e-03,   1.87685913e-02, ...,
           5.75522990e-02,   0.00000000e+00,   3.20554095e-03],
        [  5.16341685e-02,   2.03264328e-03,   1.87685913e-02, ...,
           5.75522990e-02,   0.00000000e+00,   3.20554095e-03]]),
 array([[     0.,      0.,      0., ...,   8627.,   7126.,   4726.],
        [     0.,      0.,      0., ...,   7549.,   7969.,   4726.],
        [     0.,      0.,      0., ...,  12516.,   18

In [42]:
from keras.models import model_from_json
from keras.preprocessing.sequence import pad_sequences
from nltk.tokenize import word_tokenize
from time import sleep
import numpy as np
import h5py
import json
import pickle

model_filename = '/home/ubuntu/app/resources/model.json'
model_weights_filename = '/home/ubuntu/app/resources/model_weights_xception.h5'
json_file = open(model_filename, 'r')
loaded_model_json = json_file.read()
# print "Reading Model..."
model = model_from_json(loaded_model_json)
# print "Loading Weights..."
model.load_weights(model_weights_filename)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
metadata = json.load(open('/home/ubuntu/app/resources/data_prepro.json', 'r'))
metadata['ix_to_word'] = {str(word):int(i) for i,word in metadata['ix_to_word'].items()}

def get_ques_vector(question):
    question_vector = []
    seq_length = 26
    word_index = metadata['ix_to_word']
    for word in word_tokenize(question.lower()):
        if word in word_index:
            question_vector.append(word_index[word])
        else:
            question_vector.append(0)
    question_vector = np.array(pad_sequences([question_vector], maxlen=seq_length))[0]
    question_vector = question_vector.reshape((1,seq_length))
    return question_vector





/home/ubuntu/anaconda3/lib/python3.5/site-packages/keras/engine/topology.py:1252: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(input_dtype="float32", batch_input_shape=[None, 204..., activity_regularizer=None, activation="tanh", kernel_initializer="glorot_uniform", use_bias=True, bias_constraint=None, bias_regularizer=None, kernel_constraint=None, kernel_regularizer=None, trainable=True, units=1024, name="dense_1", input_dim=4096)`
  return cls(**config)
/home/ubuntu/anaconda3/lib/python3.5/site-packages/keras/engine/topology.py:1252: UserWarning: The `dropout` argument is no longer support in `Embedding`. You can apply a `keras.layers.SpatialDropout1D` layer right after the `Embedding` layer to get the same behavior.
  return cls(**config)
/home/ubuntu/anaconda3/lib/python3.5/site-packages/keras/engine/topology.py:1252: UserWarning: Update your `Embedding` call to the Keras 2 API: `Embedding(input_dtype="float32", batch_input_shape=[None, 26], input_dim=14771, embe

In [8]:
data = h5py.File('/home/ubuntu/app/data_img.h5')

In [15]:
data['images_train'][10]

(4096,)

In [30]:

image_id = '1'

img_vector = img_data['images_test'][int(image_id)]
img_vector = img_vector.reshape((1,2048))
question_vector = get_ques_vector('what is the color?')
pred = model.predict([img_vector, question_vector])[0]
top_pred = pred.argsort()[-5:][::-1]

In [31]:
top_pred

array([11,  5,  7, 26, 13])

In [32]:
[(metadata['ix_to_ans'][str(_)].title(), round(pred[_]*100.0,2)) for _ in top_pred]

[('Black', 82.719999999999999),
 ('White', 3.8300000000000001),
 ('Red', 2.1400000000000001),
 ('Gray', 2.04),
 ('Brown', 1.99)]

In [38]:
import json
import h5py
from keras.applications.xception import Xception
from keras.applications.xception import preprocess_input
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img
import numpy as np
from keras.models import Model
from skimage import io
 

def image_preprocess(url):
    image = np.resize(io.imread(url), (299, 299,3)).astype(float)
    print(image.shape)
    # our input image is now represented as a NumPy array of shape
    # (inputShape[0], inputShape[1], 3) however we need to expand the
    # dimension by making the shape (1, inputShape[0], inputShape[1], 3)
    # so we can pass it through thenetwork
    image = np.expand_dims(image, axis=0)

    # pre-process the image using the appropriate function based on the
    # model that has been loaded (i.e., mean subtraction, scaling, etc.)
    image = preprocess_input(image)
    return image

base_model = Xception(weights='imagenet',input_shape=(299, 299, 3))
image_model = Model(inputs=base_model.input, outputs=base_model.get_layer('avg_pool').output)

In [43]:
# image_file_name = 'http://www.newarkhistory.com/indparksoccerkids.jpg'
# image = io.imread(image_file_name)

image_preprocessed = image_preprocess("http://www.newarkhistory.com/indparksoccerkids.jpg")
features = image_model.predict(image_preprocessed)

(299, 299, 3)


In [44]:
question_vector = get_ques_vector('what sport are they playing?')
pred = model.predict([features, question_vector])[0]
top_pred = pred.argsort()[-5:][::-1]

In [53]:
import cv2

In [46]:
[(metadata['ix_to_ans'][str(_)].title(), round(pred[_]*100.0,2)) for _ in top_pred][0]

('Soccer', 34.189999999999998)